In [4]:
from z3 import *
from spacer_tutorial import *

# Experiments with Scaling

## Utilities

In [17]:
# Binary Craig Interpolation by reduction to CHC
def interpolate(A, B, verbosity=0):
    '''Computes a Craig interpolant between A and B'''
    As = free_arith_vars(A)
    Bs = free_arith_vars(B)

    shared = [s for s in As & Bs ]

    Itp = z3.Function('Itp', [s.sort() for s in shared] + [z3.BoolSort()])
    left = z3.ForAll([a for a in As], z3.Implies(A, Itp(shared)))
    right = z3.ForAll([b for b in Bs], z3.Implies(Itp(shared), z3.Not(B)))

    res, answer = solve_horn([left, right])

    if res == z3.sat:
        return answer.eval(Itp(shared))
    return None 

In [39]:
def scale(formula, k):
    print(formula.len())
    print(formula)
    return formula

In [41]:
test = And(2*x + 3*y + 4 <= 0, 3*y + 2 <= 0)
scaled = scale(test, 1)
scaled

AttributeError: 'BoolRef' object has no attribute 'numArgs'

## src

In [30]:
# scale(2*x + 3*y + 4 <= 0 && 3*y + 2 <= 0, k) = 2*x + 3*y + 4k <= 0 && 3y + 2k <= 0
T = Solver()
x, x_out = Ints("x0 x1")
y, y_out = Ints("y0 y1")

A = And(y > 0)
B = And(2*x + 3*y + 4 <= 0, 3*y + 2 <= 0)
# Init = x == 0
# Tr = Or(And(x < 5, x_out == x + 1), And(x >= 453, x_out == x))
# Bad = And(x >= 6)
T.add(A)
T.add(B)
T

[And(y0 > 0), And(2*x0 + 3*y0 + 4 <= 0, 3*y0 + 2 <= 0)]

In [34]:
res = T.check()
display(res)

if res == sat:
    display(T.model())
else:
    display(interpolate(A, B))

unsat

Not(y0 <= 0)

And(2*x0 + 3*y0 + 4 <= 0, 3*y0 + 2 <= 0)


In [40]:
if res == unsat:
    B2 = scale(B, 1)

T2 = Solver()
T2.add(A)
T2.add(B2)

resScaled = T2.check()
display(resScaled)

if resScaled == sat:
    display(T2.model())
else:
    display(interpolate(A, B2))

AttributeError: 'BoolRef' object has no attribute 'numArgs'

# Experiments with IC3

In [5]:
T = Solver()
x, x_out = Ints("x0 x1")

Init = x == 0
Tr = Or(And(x < 5, x_out == x + 1), And(x >= 453, x_out == x))
Bad = And(x >= 6)

In [6]:
T = Solver()
F2 =[]
T.add(*F2)
T.add(Bad)
T.check()
T.model()

[x0 = 6]

In [7]:
# pob = x_out == T.model().eval(x)
pob = z3.substitute(Bad, (x, x_out))

# pob = [x >= 5, x >= 6]
pob

And(x1 >= 6)

In [8]:
S = Solver()
F1 = [Not(And(x >= 6))]
S.add(*F1)
S.add(Tr)
S.add(pob)
res = S.check()
if res == sat:
    display(S.model())
else:
    display(res)


unsat

In [9]:

pob_1 = z3.substitute(z3.And(x >= 6), (x, x_out))
pob_1

And(x1 >= 6)

In [62]:
S1 = Solver()
F0 = [Init]
S1.add(*F0)
S1.add(Tr)
S1.add(pob_1)
res = S1.check()
if res == sat:
    display(S.model())
else:
    display(res)

unsat

In [66]:

proxy, proxy2 = z3.Bools('proxy proxy2')

In [78]:
s = z3.Solver()
s.add(z3.Implies(proxy, z3.And(x > 0, x < 2)))
s.add(z3.Implies(proxy2, z3.And(x_out > 0, x_out < 2)))
a = z3.Bool('a')
s.add(z3.Implies(a, x > 1))
s

[Implies(proxy, And(x0 > 0, x0 < 2)),
 Implies(proxy2, And(x1 > 0, x1 < 2)),
 Implies(a, x0 > 1)]

In [80]:
s.check(proxy, z3.Not(proxy2), a)

unsat

In [79]:
s.check(proxy2, a)

sat